In [1]:
import cv2
import os
import glob
import random
import shutil
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

from ultralytics import YOLO
import torch

In [ ]:
# Load your custom model configuration
model = YOLO('yolo11n-bifpn.yaml')

# Print info to verify layers are there
model.info()

YOLO11n-bifpn summary: 200 layers, 2,599,933 parameters, 2,599,917 gradients, 6.6 GFLOPs


(200, 2599933, 2599917, 6.590592000000001)

In [8]:
def train_tuned_model():

    # Using the Nano architecture
    model = YOLO('yolo11n-bifpn.yaml') 

    # 2. Load Pre-trained weights
    try:
        if os.path.exists('yolo11n.pt'):
            model.load('yolo11n.pt')
    except:
        pass

    # 3. Advanced Training Configuration
    model.train(
        data='data.yaml',      
        project='BrainTumor_BiFPN_Multiclass', 
        name='run_tuned_v3',      
        
        # --- Training Dynamics ---
        epochs=100,               
        patience=15,              
        batch=16,            
        imgsz=640,
        device=0,
        workers=4,               
        optimizer='AdamW',        
        lr0=0.001,                
        cos_lr=True,              
        
        # --- DATA AUGMENTATION ---
        # Keep the same settings that worked well for medical images
        # degrees=15.0,             
        # flipud=0.5,               
        # fliplr=0.5,               
        # mosaic=1.0,               
        # mixup=0.1,                
        
        # --- Optimization ---
        amp=True,                 
        exist_ok=True
    )
    
    print("Tuned Training V3 Complete!")
    
    # 4. Validate with TTA
    print("\nRunning Validation with TTA...")
    metrics = model.val(data='data.yaml', split='val', augment=True)
    print(f"TTA mAP50: {metrics.box.map50}")

if __name__ == '__main__':
    torch.multiprocessing.freeze_support()
    train_tuned_model()

Transferred 245/548 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.234 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.11.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-bifpn.yaml, momentum=0.937, mosaic=1.0, multi_s

In [9]:
# 1. Define the path to your BEST Tuned Model (Run V2)
# This path matches the output from your successful training log
model_path = "BrainTumor_BiFPN_Multiclass/run_tuned_v3/weights/best.pt"

if not os.path.exists(model_path):
    print(f"Error: Weights not found at: {model_path}")
else:
    print(f"Found Model: {model_path}")
    
    # 2. Load the model
    model = YOLO(model_path)
    
    # 3. Verify Classes (Should be 3: glioma, meningioma, pituitary)
    print(f"   Classes detected: {model.names}")
    
    # 4. Export to ONNX
    print("\nExport to ONNX...")
    export_path = model.export(format='onnx', opset=12, dynamic=False)
    
    print(f"ONNX File: {export_path}")

Found Model: BrainTumor_BiFPN_Multiclass/run_tuned_v3/weights/best.pt
   Classes detected: {0: 'glioma', 1: 'meningioma', 2: 'pituitary'}

Export to ONNX...
Ultralytics 8.3.233  Python-3.11.14 torch-2.5.1 CPU (AMD Ryzen 7 4800H with Radeon Graphics)
YOLO11n-bifpn summary (fused): 112 layers, 2,591,867 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'BrainTumor_BiFPN_Multiclass\run_tuned_v3\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.3 MB)

ONNX: starting export with onnx 1.19.1 opset 12...
ONNX: slimming with onnxslim 0.1.77...
ONNX: export success  2.0s, saved as 'BrainTumor_BiFPN_Multiclass\run_tuned_v3\weights\best.onnx' (10.2 MB)

Export complete (2.5s)
Results saved to D:\Data Adji\Binus\Semester 5\DeepLearning\Project\BrainTumor_BiFPN_Multiclass\run_tuned_v3\weights
Predict:         yolo predict task=detect model=BrainTumor_BiFPN_Multiclass\run_tuned_v3\weights\best.onnx imgsz=640  
Validate:        yolo val task=detect 

In [12]:
# --- CONFIGURATION ---
# Point to your BEST Tuned Model (Run V2)
MODEL_PATH = "BrainTumor_BiFPN_Multiclass/run_tuned_v3/weights/best.pt"
# Base path for your classification test set
TEST_DATA_ROOT = "brisc2025/classification_task/test"

# Classes to test
CLASSES_TO_TEST = ['glioma', 'meningioma', 'pituitary', 'no_tumor']

def run_visual_test():
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: Weights not found at: {MODEL_PATH}")
        return

    print(f"✅ Loading Model: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)

    # Loop through each class to pick one random image
    for i, class_name in enumerate(CLASSES_TO_TEST):
        class_dir = os.path.join(TEST_DATA_ROOT, class_name)
        
        # Find images (jpg or png)
        images = glob.glob(os.path.join(class_dir, '*.jpg')) + glob.glob(os.path.join(class_dir, '*.png'))
        
        if not images:
            print(f"⚠️ No images found for class '{class_name}' in {class_dir}")
            continue

        # Pick a random image
        img_path = random.choice(images)
        print(f"Testing {class_name}: {os.path.basename(img_path)}")
        
        # Run Inference
        # conf=0.25 is a standard threshold for visualization
        results = model.predict(img_path, conf=0.25, verbose=False)
        result = results[0] # Get first result
        
        # Prepare Title / Overlay Text
        if len(result.boxes) == 0:
            pred_text = "No Detection"
            # Green if correct for no_tumor, Red if missed tumor
            color = (0, 255, 0) if class_name == 'no_tumor' else (0, 0, 255)
        else:
            # Get top prediction
            box = result.boxes[0]
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            pred_class = result.names[cls_id]
            
            pred_text = f"Pred: {pred_class} ({conf:.1%})"
            
            # Check if correct (Loose check based on folder name)
            if class_name == 'no_tumor':
                 color = (0, 0, 255) # Red - False Positive
            elif class_name == pred_class:
                 color = (0, 255, 0) # Green - Correct
            else:
                 color = (0, 165, 255) # Orange - Wrong Class

        # Draw on image
        # result.plot() returns a BGR numpy array (perfect for OpenCV)
        res_plotted = result.plot()
        
        # Add text overlay
        # Line 1: True Class
        cv2.putText(res_plotted, f"True: {class_name}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        # Line 2: Prediction
        cv2.putText(res_plotted, pred_text, (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Display Window
        window_name = f"Test {i+1}: {class_name}"
        cv2.imshow(window_name, res_plotted)

    print("\nTest Complete! Press any key to close the image windows.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_visual_test()

✅ Loading Model: BrainTumor_BiFPN_Multiclass/run_tuned_v3/weights/best.pt
Testing glioma: brisc2025_test_00253_gl_sa_t1.jpg
Testing meningioma: brisc2025_test_00527_me_sa_t1.jpg
Testing pituitary: brisc2025_test_00776_pi_ax_t1.jpg
Testing no_tumor: brisc2025_test_00640_no_co_t1.jpg

Test Complete! Press any key to close the image windows.
